<a href="https://colab.research.google.com/github/4graf/Data_Processing/blob/main/ML/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.svm import LinearSVC
import cv2

In [ ]:
# ! unzip test_task.zip
# ! unzip train_task.zip

In [ ]:
from os import listdir

imgs_train = sorted(['train/' + name for name in listdir('train')])[1:]
imgs_test = sorted(['test/' + name for name in listdir('test')])[2:]

df_train = pd.DataFrame({'Image': imgs_train})
df_test = pd.DataFrame({'Image': imgs_test})

df_train['Y'] = np.where(df_train['Image'].str.contains('dog'), 1, 0)
df_test['Y'] = np.where(df_test['Image'].str.contains('dog'), 1, 0)
df_train

,Image,Y
0,train/cat.0.jpg,0
1,train/cat.1.jpg,0
2,train/cat.10.jpg,0
3,train/cat.100.jpg,0
4,train/cat.101.jpg,0
...,...,...
995,train/dog.95.jpg,1
996,train/dog.96.jpg,1
997,train/dog.97.jpg,1
998,train/dog.98.jpg,1


In [ ]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
from sklearn.model_selection import train_test_split 

svc = LinearSVC(C=1.47, random_state=4)
X_data = list(map(lambda x: extract_histogram(cv2.imread(x)), df_train['Image']))
y_data = df_train['Y']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    random_state=4, train_size=0.75)

svc.fit(X_train, y_train)

LinearSVC(C=1.47, random_state=4)

In [ ]:
print(f'{svc.coef_[0][66]:.10f}')

-1.0325092019


In [ ]:
y_predict = svc.predict(X_test)
print('Matrix confusion:\n', metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))
# metrics.roc_curve(y_test, )

Matrix confusion:
 [[86 41]
 [67 56]]
              precision    recall  f1-score   support

           0       0.56      0.68      0.61       127
           1       0.58      0.46      0.51       123

    accuracy                           0.57       250
   macro avg       0.57      0.57      0.56       250
weighted avg       0.57      0.57      0.56       250



In [ ]:
# svc.fit(X_data, y_data)

X_new = list(map(lambda x: extract_histogram(cv2.imread(x)), df_test['Image']))
y_new = df_test['Y']
df_test['predict'] = svc.predict(X_new)
df_test.loc[20]

Image      test/cat.1020.jpg
Y                          0
predict                    0
Name: 20, dtype: object